In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import re

class SimpleChatbot:
    def __init__(self, model_name="microsoft/DialoGPT-medium"):
        # 加载预训练模型和分词器
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        
        # 设置模型为评估模式
        self.model.eval()
        
        # 初始化聊天历史
        self.chat_history_ids = None
        
        # 设置生成参数
        self.generation_params = {
            "max_length": 1000,
            "pad_token_id": self.tokenizer.eos_token_id,
            "no_repeat_ngram_size": 3,
            "do_sample": True,
            "top_k": 100,
            "top_p": 0.9,
            "temperature": 0.8
        }
    
    def generate_response(self, user_input):
        # 对用户输入进行编码
        new_input_ids = self.tokenizer.encode(
            user_input + self.tokenizer.eos_token, 
            return_tensors="pt"
        )
        
        # 将新输入与聊天历史合并
        if self.chat_history_ids is not None:
            bot_input_ids = torch.cat([self.chat_history_ids, new_input_ids], dim=-1)
        else:
            bot_input_ids = new_input_ids
        
        # 生成响应
        with torch.no_grad():
            self.chat_history_ids = self.model.generate(
                bot_input_ids,
                **self.generation_params
            )
        
        # 提取最新响应
        response_start_idx = bot_input_ids.shape[-1]
        response_ids = self.chat_history_ids[:, response_start_idx:]
        
        # 解码响应
        response = self.tokenizer.decode(
            response_ids[0], 
            skip_special_tokens=True
        )
        
        # 清理响应中的多余空格
        response = re.sub(r'\s+', ' ', response).strip()
        
        return response
    
    def reset_chat(self):
        """重置聊天历史"""
        self.chat_history_ids = None

def main():
    # 初始化聊天机器人
    print("正在加载聊天机器人，请稍候...")
    chatbot = SimpleChatbot()
    print("聊天机器人已就绪! 输入'退出'来结束对话。")
    
    # 开始对话循环
    while True:
        user_input = input("你: ")
        
        if user_input.lower() in ['退出', 'exit', 'quit']:
            print("再见!")
            break
        
        # 生成并显示响应
        response = chatbot.generate_response(user_input)
        print(f"机器人: {response}")
        
        # 可选：每5轮对话后重置历史以避免过长
        # 在实际应用中可能需要更复杂的策略

if __name__ == "__main__":
    main()

c:\Users\wind\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


正在加载聊天机器人，请稍候...


OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Check your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.